<a href="https://colab.research.google.com/github/makaryb/nn2s5k/blob/master/kurs/cats_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# get acess google drive data into google colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from zipfile import ZipFile
filename = "/content/data7/dataset.zip"
with ZipFile(filename, "r") as zip:
  zip.extractall()
  print("finish")

finish


In [1]:
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import cv2
from keras.layers import Activation, Dense, Conv2D, UpSampling2D, LeakyReLU, Reshape, Flatten, Input, BatchNormalization, Dropout

Using TensorFlow backend.


In [0]:
def build_discriminator(img_shape):
    model = Sequential()  # 64x64

    model.add(Conv2D(32, kernel_size=5, strides=2, input_shape=img_shape, padding="same"))  # 32x32
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=5, strides=2, padding="same"))  # 16x16
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))  # 8x8
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    model.summary()
    img = Input(shape=img_shape)
    d_pred = model(img)
    return Model(input=img, output=d_pred)

def build_generator(z_dimension, channels):
    model = Sequential()

    model.add(Dense(128 * 8 * 8, input_dim=z_dimension))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8, 8, 128)))

    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=5, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=5, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(UpSampling2D())
    model.add(Conv2D(32, kernel_size=5, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels, kernel_size=5, padding="same"))
    model.add(Activation("tanh"))

    model.summary()
    noise = Input(shape=(z_dimension,))
    img = model(noise)
    return Model(input=noise, output=img)

def sample_images(epoch):
    r, c = 4, 5
    noise = np.random.normal(0, 1, (r * c, z_dimension))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(PATH + "output64_cat/n_%d.png" % epoch, dpi=200)
    plt.close()

In [0]:
# dataset
imaglist = []
for i in range(9000):
    imag = cv2.imread("out_unaug_64x64/{num}_000.jpg".format(num=str(i).zfill(6)))
    imag = cv2.cvtColor(imag, cv2.COLOR_BGR2RGB)
    imaglist.append(imag)

x_train = np.array(imaglist)
x_train = x_train / 127.5 - 1.  # values -1 to 1

In [0]:
# parameters
PATH = ""
img_rows = 64
img_cols = 64
channels = 3
img_shape = (img_rows, img_cols, channels)
z_dimension = 64
optimizer = Adam(0.0005, 0.5)

In [0]:
# discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

In [0]:
# generator
generator = build_generator(z_dimension, channels)

In [0]:
# the generator takes noise as input and generates imgs
z = Input(shape=(z_dimension,))
img = generator(z)
discriminator.trainable = False
d_pred = discriminator(img)
# The combined model  (stacked generator and discriminator)
combined = Model(z, d_pred)
combined.compile(loss='binary_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [0]:
# training parameters
epochs = 10000
batch_size = 64
sample_interval = 100  # save some generated pictrures

In [0]:
# adversarial ground truths
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

In [0]:
# training
for epoch in range(epochs):
    print("epoch: ", epoch)
    # real images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]
    # generated images
    noise = np.random.normal(0, 1, (batch_size, z_dimension))
    gen_imgs = generator.predict(noise)
    # train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, real)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    #d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    # train generator
    noise = np.random.normal(0, 1, (batch_size, z_dimension))
    g_loss = combined.train_on_batch(noise, real)
    # save progress
    if (epoch % sample_interval) == 0:
        print("%d [D loss on real: %f, D loss on fake: %f, acc. on real: %.2f%%, acc. on fake: %.2f%%] [G loss: %f]" %
              (epoch, d_loss_real[0], d_loss_fake[0], 100 * d_loss_real[1], 100 * d_loss_fake[1], g_loss[0]))
        sample_images(epoch)
        generator.save("n_generator_64_64_z64_%d_epoch.h5" % epoch)

### Задание, поставленное в данной работе: необходимо создать модель генеративной нейронной сети для генерирования изображений котов и обучить эту модель за счет создания и обучения модели генеративно-состязательной нейронной сети (GAN).